Лучшие бюджетные игровые сборки

In [ ]:
# Ryzen 5 5500;
# AM4, 6 core, 3.6-4.2 ГГЦ, PCIE 3.0
# брать видеокарты не старше 30 серии и не старше 60 линейки

low_cost_best = {}
low_cost_best[111] = {'cpu': ('ryzen 5 5500', 'AM4', 'PCIE 3.0', '6 core', '3.6-4.2 GGC', 9900), 'mb' : ('MSI A520A-M PRO', '2xRAM 3200 GGC', 'A520', 'PCIE 3.0', 6070), 'gpu': ('GIGABYTE GeForce RTX 2060 D6', '6 GD', 'PCIE 3.0', 17899)}

# Ryzen 5 5600;
# AM4, 6 core, 3.5-4.4 ГГЦ, PCIE 4.0
# брать видеокарты не старше 60 линейки
# аналог Ryzen 5 5500, но шина для видеокарты новее, поэтому можно использовать все серии видеокарт

low_cost_best[222] = {'cpu': ('ryzen 5 5600', 'AM4', 'PCIE 4.0', '6 core', '3.5-4.4 GGC', 12990), 'mb' : ('MSI PRO H610M-E', '2xRAM 3200 GGC', 'B550', 'PCIE 4.0', 9380), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 3060 GV-N3060WF2OC-12GD 2.0', '12 GD', 'PCIE 4.0', 37910)}
low_cost_best[223] = {'cpu': ('ryzen 5 5600', 'AM4', 'PCIE 4.0', '6 core', '3.5-4.4 GGC', 12990), 'mb' : ('MSI PRO H610M-E', '2xRAM 3200 GGC', 'B550', 'PCIE 4.0', 9380), 'gpu': ('Palit NVIDIA GeForce RTX 4060 RTX4060 DUAL', '8 GD', 'PCIE 4.0', 39300)}

# Intel i5 12400F;
# LGA 1700, 6 core, 2.5-4.4 ГГЦ, PCIE 4.0
# брать видеокарты не старше 70 линейки
# дороговат но можно замахнуться на видеокарты 70 линейки Игровой товар

low_cost_best[332] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('ASROCK H610M-HDV', '2xRAM 3200 GGC', 'H610', 'PCIE 4.0', 7440), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 3060 GV-N3060WF2OC-12GD 2.0', '12 GD', 'PCIE 4.0', 37910)}
low_cost_best[333] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('ASROCK H610M-HDV', '2xRAM 3200 GGC', 'H610', 'PCIE 4.0', 7440), 'gpu': ('Palit NVIDIA GeForce RTX 4060 RTX4060 DUAL', '8 GD', 'PCIE 4.0', 39300)}
low_cost_best[342] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('MSI PRO H610M-E', '2xRAM 5600 GGC', 'H610', 'PCIE 4.0', 8040), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 3060 GV-N3060WF2OC-12GD 2.0', '12 GD', 'PCIE 4.0', 37910)}
low_cost_best[342] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('MSI PRO H610M-E', '2xRAM 5600 GGC', 'H610', 'PCIE 4.0', 8040), 'gpu': ('Palit NVIDIA GeForce RTX 4060 RTX4060 DUAL', '8 GD', 'PCIE 4.0', 39300)}
low_cost_best[342] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('MSI PRO H610M-E', '2xRAM 5600 GGC', 'H610', 'PCIE 4.0', 8040), 'gpu': ('INNO3D GeForce RTX 4070 TWIN X2', '12 GD', 'PCIE 4.0', 63999)}

Игровые сборки, которые при малом бюджете на обновление не требуют апгрейда (рекомендуется для будующего апгрейда заменить материнскую плату)

Недостаток приведённых ниже сборок заключается в использовании либо слишком дорогих, либо слишком ненадёжных видеокарт

In [ ]:
low_cost_normal = {}

# Ryzen 5 5500
# Отличается не самым лучшим выбором видеокарты,
# она нормальная, но, как правило неоправданно дорогая
# в данном случае, если пользователь будет иметь малую
# сумму для модернизации, то нужно предложить ему оставить деньги
# для полной пересборки ПК в будущем

low_cost_normal[111] = {'cpu': ('ryzen 5 5500', 'AM4', 'PCIE 3.0', '6 core', '3.6-4.2 GGC', 9900), 'mb' : ('MSI A520A-M PRO', '2xRAM 3200 GGC', 'A520', 'PCIE 3.0', 6070), 'gpu': ('MSI NVIDIA GeForce RTX 2060 RTX 2060 VENTUS GP OC', '6 GD', 'PCIE 3.0', 39900)}

# Ryzen 5600
# Отличается не самым лучшим выбором видеокарты,
# она нормальная, но, как правило неоправданно дорогая, либо не очень надёжная
# в данном случае, если пользователь будет иметь малую
# сумму для модернизации, то нужно предложить ему оставить деньги
# для полной пересборки ПК в будущем

low_cost_normal[222] = {'cpu': ('ryzen 5 5600', 'AM4', 'PCIE 4.0', '6 core', '3.5-4.4 GGC', 12990), 'mb' : ('MSI PRO H610M-E', '2xRAM 3200 GGC', 'B550', 'PCIE 4.0', 9380), 'gpu': ('Palit NVIDIA GeForce RTX 3060 PA-RTX3060 STORMX 8GB', '8 GD', 'PCIE 4.0', 34590)}
low_cost_normal[223] = {'cpu': ('ryzen 5 5600', 'AM4', 'PCIE 4.0', '6 core', '3.5-4.4 GGC', 12990), 'mb' : ('MSI PRO H610M-E', '2xRAM 3200 GGC', 'B550', 'PCIE 4.0', 9380), 'gpu': ('MSI NVIDIA GeForce RTX 3060 RTX 3060 VENTUS 2X 12G OC', '12 GD', 'PCIE 4.0', 41260)}
low_cost_normal[224] = {'cpu': ('ryzen 5 5600', 'AM4', 'PCIE 4.0', '6 core', '3.5-4.4 GGC', 12990), 'mb' : ('MSI PRO H610M-E', '2xRAM 3200 GGC', 'B550', 'PCIE 4.0', 9380), 'gpu': ('ASUS NVIDIA GeForce RTX 3060 DUAL-RTX3060-O12G-V2 LHR', '12 GD', 'PCIE 4.0', 47490)}
low_cost_normal[225] = {'cpu': ('ryzen 5 5600', 'AM4', 'PCIE 4.0', '6 core', '3.5-4.4 GGC', 12990), 'mb' : ('MSI PRO H610M-E', '2xRAM 3200 GGC', 'B550', 'PCIE 4.0', 9380), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 3060 GV-N3060GAMING OC-8GD', '8 GD', 'PCIE 4.0', 38050)}
low_cost_normal[226] = {'cpu': ('ryzen 5 5600', 'AM4', 'PCIE 4.0', '6 core', '3.5-4.4 GGC', 12990), 'mb' : ('MSI PRO H610M-E', '2xRAM 3200 GGC', 'B550', 'PCIE 4.0', 9380), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 4060 GV-N4060D6-8GD', '8 GD', 'PCIE 4.0', 40200)}
low_cost_normal[227] = {'cpu': ('ryzen 5 5600', 'AM4', 'PCIE 4.0', '6 core', '3.5-4.4 GGC', 12990), 'mb' : ('MSI PRO H610M-E', '2xRAM 3200 GGC', 'B550', 'PCIE 4.0', 9380), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 4060 GV-N4060OC-8GL', '8 GD', 'PCIE 4.0', 42880)}
low_cost_normal[228] = {'cpu': ('ryzen 5 5600', 'AM4', 'PCIE 4.0', '6 core', '3.5-4.4 GGC', 12990), 'mb' : ('MSI PRO H610M-E', '2xRAM 3200 GGC', 'B550', 'PCIE 4.0', 9380), 'gpu': ('MSI NVIDIA GeForce RTX 4060 RTX 4060 VENTUS 3X 8G OC', '8 GD', 'PCIE 4.0', 43500)}
low_cost_normal[229] = {'cpu': ('ryzen 5 5600', 'AM4', 'PCIE 4.0', '6 core', '3.5-4.4 GGC', 12990), 'mb' : ('MSI PRO H610M-E', '2xRAM 3200 GGC', 'B550', 'PCIE 4.0', 9380), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 4060 GV-N4060EAGLE OC-8GD', '8 GD', 'PCIE 4.0', 43530)}
low_cost_normal[2210] = {'cpu': ('ryzen 5 5600', 'AM4', 'PCIE 4.0', '6 core', '3.5-4.4 GGC', 12990), 'mb' : ('MSI PRO H610M-E', '2xRAM 3200 GGC', 'B550', 'PCIE 4.0', 9380), 'gpu': ('MSI NVIDIA GeForce RTX 4060 RTX 4060 GAMING X 8G', '8 GD', 'PCIE 4.0', 43970)}
low_cost_normal[2211] = {'cpu': ('ryzen 5 5600', 'AM4', 'PCIE 4.0', '6 core', '3.5-4.4 GGC', 12990), 'mb' : ('MSI PRO H610M-E', '2xRAM 3200 GGC', 'B550', 'PCIE 4.0', 9380), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 4060 GV-N4060WF2OC-8GD', '8 GD', 'PCIE 4.0', 44640)}
low_cost_normal[2212] = {'cpu': ('ryzen 5 5600', 'AM4', 'PCIE 4.0', '6 core', '3.5-4.4 GGC', 12990), 'mb' : ('MSI PRO H610M-E', '2xRAM 3200 GGC', 'B550', 'PCIE 4.0', 9380), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 4060 GV-N4060EAGLEOC ICE-8GD', '8 GD', 'PCIE 4.0', 44480)}
low_cost_normal[2213] = {'cpu': ('ryzen 5 5600', 'AM4', 'PCIE 4.0', '6 core', '3.5-4.4 GGC', 12990), 'mb' : ('MSI PRO H610M-E', '2xRAM 3200 GGC', 'B550', 'PCIE 4.0', 9380), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 4060 GV-N4060GAMING OC-8GD', '8 GD', 'PCIE 4.0', 45920)}
low_cost_normal[2214] = {'cpu': ('ryzen 5 5600', 'AM4', 'PCIE 4.0', '6 core', '3.5-4.4 GGC', 12990), 'mb' : ('MSI PRO H610M-E', '2xRAM 3200 GGC', 'B550', 'PCIE 4.0', 9380), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 4060 GV-N4060AERO OC-8GD', '8 GD', 'PCIE 4.0', 46790)}
low_cost_normal[2215] = {'cpu': ('ryzen 5 5600', 'AM4', 'PCIE 4.0', '6 core', '3.5-4.4 GGC', 12990), 'mb' : ('MSI PRO H610M-E', '2xRAM 3200 GGC', 'B550', 'PCIE 4.0', 9380), 'gpu': ('ASUS NVIDIA GeForce RTX 4060 PROART-RTX4060-O8G', '8 GD', 'PCIE 4.0', 48860)}

# Intel i5 12400F
# Отличается не самым лучшим выбором видеокарты,
# она нормальная, но, как правило неоправданно дорогая, либо не очень надёжная
# в данном случае, если пользователь будет иметь малую
# сумму для модернизации, то нужно предложить ему оставить деньги
# для полной пересборки ПК в будущем

low_cost_normal[332] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('ASROCK H610M-HDV', '2xRAM 3200 GGC', 'H610', 'PCIE 4.0', 7440), 'gpu': ('Palit NVIDIA GeForce RTX 3060 PA-RTX3060 STORMX 8GB', '8 GD', 'PCIE 4.0', 34590)}
low_cost_normal[333] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('ASROCK H610M-HDV', '2xRAM 3200 GGC', 'H610', 'PCIE 4.0', 7440), 'gpu': ('MSI NVIDIA GeForce RTX 3060 RTX 3060 VENTUS 2X 12G OC', '12 GD', 'PCIE 4.0', 41260)}
low_cost_normal[334] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('ASROCK H610M-HDV', '2xRAM 3200 GGC', 'H610', 'PCIE 4.0', 7440), 'gpu': ('ASUS NVIDIA GeForce RTX 3060 DUAL-RTX3060-O12G-V2 LHR', '12 GD', 'PCIE 4.0', 47490)}
low_cost_normal[335] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('ASROCK H610M-HDV', '2xRAM 3200 GGC', 'H610', 'PCIE 4.0', 7440), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 3060 GV-N3060GAMING OC-8GD', '8 GD', 'PCIE 4.0', 38050)}
low_cost_normal[336] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('ASROCK H610M-HDV', '2xRAM 3200 GGC', 'H610', 'PCIE 4.0', 7440), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 4060 GV-N4060D6-8GD', '8 GD', 'PCIE 4.0', 40200)}
low_cost_normal[337] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('ASROCK H610M-HDV', '2xRAM 3200 GGC', 'H610', 'PCIE 4.0', 7440), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 4060 GV-N4060OC-8GL', '8 GD', 'PCIE 4.0', 42880)}
low_cost_normal[338] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('ASROCK H610M-HDV', '2xRAM 3200 GGC', 'H610', 'PCIE 4.0', 7440), 'gpu': ('MSI NVIDIA GeForce RTX 4060 RTX 4060 VENTUS 3X 8G OC', '8 GD', 'PCIE 4.0', 43500)}
low_cost_normal[339] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('ASROCK H610M-HDV', '2xRAM 3200 GGC', 'H610', 'PCIE 4.0', 7440), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 4060 GV-N4060EAGLE OC-8GD', '8 GD', 'PCIE 4.0', 43530)}
low_cost_normal[3310] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('ASROCK H610M-HDV', '2xRAM 3200 GGC', 'H610', 'PCIE 4.0', 7440), 'gpu': ('MSI NVIDIA GeForce RTX 4060 RTX 4060 GAMING X 8G', '8 GD', 'PCIE 4.0', 43970)}
low_cost_normal[3311] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('ASROCK H610M-HDV', '2xRAM 3200 GGC', 'H610', 'PCIE 4.0', 7440), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 4060 GV-N4060WF2OC-8GD', '8 GD', 'PCIE 4.0', 44640)}
low_cost_normal[3312] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('ASROCK H610M-HDV', '2xRAM 3200 GGC', 'H610', 'PCIE 4.0', 7440), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 4060 GV-N4060EAGLEOC ICE-8GD', '8 GD', 'PCIE 4.0', 44480)}
low_cost_normal[3313] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('ASROCK H610M-HDV', '2xRAM 3200 GGC', 'H610', 'PCIE 4.0', 7440), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 4060 GV-N4060GAMING OC-8GD', '8 GD', 'PCIE 4.0', 45920)}
low_cost_normal[3314] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('ASROCK H610M-HDV', '2xRAM 3200 GGC', 'H610', 'PCIE 4.0', 7440), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 4060 GV-N4060AERO OC-8GD', '8 GD', 'PCIE 4.0', 46790)}
low_cost_normal[3315] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('ASROCK H610M-HDV', '2xRAM 3200 GGC', 'H610', 'PCIE 4.0', 7440), 'gpu': ('ASUS NVIDIA GeForce RTX 4060 PROART-RTX4060-O8G', '8 GD', 'PCIE 4.0', 48860)}
low_cost_normal[3416] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('MSI PRO H610M-E', '2xRAM 5600 GGC', 'H610', 'PCIE 4.0', 8040), 'gpu': ('MSI NVIDIA GeForce RTX 4070 RTX 4070 VENTUS 2X E 12G', '12 GD', 'PCIE 4.0', 75130)}
low_cost_normal[3417] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('MSI PRO H610M-E', '2xRAM 5600 GGC', 'H610', 'PCIE 4.0', 8040), 'gpu': ('MSI NVIDIA GeForce RTX 4070 Super RTX 4070 SUPER 12G', '12 GD', 'PCIE 4.0', 79600)}
low_cost_normal[3418] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('MSI PRO H610M-E', '2xRAM 5600 GGC', 'H610', 'PCIE 4.0', 8040), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 4070 GV-N4070WF3-12GD', '12 GD', 'PCIE 4.0', 79780)}
low_cost_normal[3419] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('MSI PRO H610M-E', '2xRAM 5600 GGC', 'H610', 'PCIE 4.0', 8040), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 4070 GV-N4070WF3OC-12GD', '12 GD', 'PCIE 4.0', 79950)}
low_cost_normal[3420] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('MSI PRO H610M-E', '2xRAM 5600 GGC', 'H610', 'PCIE 4.0', 8040), 'gpu': ('Palit NVIDIA GeForce RTX 4070 Super RTX4070 SUPER DUAL OC', '12 GD', 'PCIE 4.0', 82480)}
low_cost_normal[3421] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('MSI PRO H610M-E', '2xRAM 5600 GGC', 'H610', 'PCIE 4.0', 8040), 'gpu': ('ASUS NVIDIA GeForce RTX 4070 DUAL-RTX4070-O12G', '12 GD', 'PCIE 4.0', 82950)}
low_cost_normal[3422] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('MSI PRO H610M-E', '2xRAM 5600 GGC', 'H610', 'PCIE 4.0', 8040), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 4070 GV-N4070EAGLE OCV2-12GD', '12 GD', 'PCIE 4.0', 83900)}
low_cost_normal[3423] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('MSI PRO H610M-E', '2xRAM 5600 GGC', 'H610', 'PCIE 4.0', 8040), 'gpu': ('Palit NVIDIA GeForce RTX 4070 RTX4070 DUAL', '12 GD', 'PCIE 4.0', 79990)}
low_cost_normal[3424] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('MSI PRO H610M-E', '2xRAM 5600 GGC', 'H610', 'PCIE 4.0', 8040), 'gpu': ('ASUS NVIDIA GeForce RTX 4070 DUAL-RTX4070-O12G-WHITE', '12 GD', 'PCIE 4.0', 84430)}
low_cost_normal[3425] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('MSI PRO H610M-E', '2xRAM 5600 GGC', 'H610', 'PCIE 4.0', 8040), 'gpu': ('Palit NVIDIA GeForce RTX 4070 PA-RTX4070 JETSTREAM', '12 GD', 'PCIE 4.0', 84540)}
low_cost_normal[3426] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('MSI PRO H610M-E', '2xRAM 5600 GGC', 'H610', 'PCIE 4.0', 8040), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 4070 GV-N4070EAGLE OC-12GD', '12 GD', 'PCIE 4.0', 86600)}
low_cost_normal[3427] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('MSI PRO H610M-E', '2xRAM 5600 GGC', 'H610', 'PCIE 4.0', 8040), 'gpu': ('ASUS NVIDIA GeForce RTX 4070 PROART-RTX4070-O12G', '12 GD', 'PCIE 4.0', 88130)}
low_cost_normal[3428] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('MSI PRO H610M-E', '2xRAM 5600 GGC', 'H610', 'PCIE 4.0', 8040), 'gpu': ('ASUS NVIDIA GeForce RTX 4070TI ROG-STRIX-RTX4070TI-O12G-GAMING', '12 GD', 'PCIE 4.0', 134710)}
low_cost_normal[3429] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('MSI PRO H610M-E', '2xRAM 5600 GGC', 'H610', 'PCIE 4.0', 8040), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 4070TI Super GV-N407TSEAGLEOCICE-16GD', '12 GD', 'PCIE 4.0', 123740)}
low_cost_normal[3430] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('MSI PRO H610M-E', '2xRAM 5600 GGC', 'H610', 'PCIE 4.0', 8040), 'gpu': ('ASUS NVIDIA GeForce RTX 4070TI TUF-RTX4070TI-12G-GAMING', '12 GD', 'PCIE 4.0', 122050)}
low_cost_normal[3431] = {'cpu': ('intel i5 12400F', 'LGA1700', 'PCIE 4.0', '6 core', '2.5-4.4 GGC', 15790), 'mb' : ('MSI PRO H610M-E', '2xRAM 5600 GGC', 'H610', 'PCIE 4.0', 8040), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 4070TI Super GV-N407TSAERO OC-16GD', '12 GD', 'PCIE 4.0', 121600)}

In [ ]:
budget_pc = {}
budget_pc['1111'] = {'mb': ('MSI A320M-A PRO', 6240),        'cpu': ('ryzen 5 5500', 9490), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 3060', 36000), 'op': ('Kingston Fury Beast Black DDR4 2x8 2666', 5090)}


budget_pc['2111'] = {'mb': ('GIGABYTE A520M K V2', 6440),    'cpu': ('ryzen 5 5500', 9490), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 3060', 36000), 'op': ('Patriot Viper 4 Blackout DDR4 2x8 3200', 4190)}
budget_pc['2112'] = {'mb': ('GIGABYTE A520M K V2', 6440),    'cpu': ('ryzen 5 5500', 9490), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 3060', 36000), 'op': ('Patriot Viper Steel DDR4 2x8 3200', 4190)}
budget_pc['2113'] = {'mb': ('GIGABYTE A520M K V2', 6440),    'cpu': ('ryzen 5 5500', 9490), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 3060', 36000), 'op': ('Kingston Fury Beast Black DDR4 2x8 3200', 4590)}
budget_pc['2114'] = {'mb': ('GIGABYTE A520M K V2', 6440),    'cpu': ('ryzen 5 5500', 9490), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 3060', 36000), 'op': ('Kingston Fury Beast RGB DDR4 2x8 3200', 5990)}


budget_pc['3111'] = {'mb': ('MSI B450M-A PRO MAX II', 6970), 'cpu': ('ryzen 5 5500', 9490), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 3060', 36000), 'op': ('Patriot Viper Elite II DDR4 2x8 4000', 5090)}
budget_pc['3112'] = {'mb': ('MSI B450M-A PRO MAX II', 6970), 'cpu': ('ryzen 5 5500', 9490), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 3060', 36000), 'op': ('Patriot Viper 4 Blackout DDR4 2x8 4000', 7990)}
budget_pc['3113'] = {'mb': ('MSI B450M-A PRO MAX II', 6970), 'cpu': ('ryzen 5 5500', 9490), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 3060', 36000), 'op': ('Kingston Fury Renegade RGB DDR4 2x8 4000', 8990)}


budget_pc['4111'] = {'mb': ('ASROCK H610M-HVS', 7230), 'cpu': ('Intel Core i5 12400F', 14790), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 3060', 36000), 'op': ('Patriot Viper 4 Blackout DDR4 2x8 3200', 4190)}
budget_pc['4112'] = {'mb': ('ASROCK H610M-HVS', 7230), 'cpu': ('Intel Core i5 12400F', 14790), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 3060', 36000), 'op': ('Patriot Viper Steel DDR4 2x8 3200', 4190)}
budget_pc['4113'] = {'mb': ('ASROCK H610M-HVS', 7230), 'cpu': ('Intel Core i5 12400F', 14790), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 3060', 36000), 'op': ('Kingston Fury Beast Black DDR4 2x8 3200', 4590)}
budget_pc['4114'] = {'mb': ('ASROCK H610M-HVS', 7230), 'cpu': ('Intel Core i5 12400F', 14790), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 3060', 36000), 'op': ('Kingston Fury Beast RGB DDR4 2x8 3200', 5990)}

budget_pc['5111'] = {'mb': ('MSI PRO H610M-E', 8370), 'cpu': ('Intel Core i5 12400F', 14790), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 3060', 36000), 'op': ('Patriot Viper Elite II DDR4 2x8 4000', 5090)}
budget_pc['5111'] = {'mb': ('MSI PRO H610M-E', 8370), 'cpu': ('Intel Core i5 12400F', 14790), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 3060', 36000), 'op': ('Patriot Viper Venom DDR5 2x8 5600', 6990)}
budget_pc['5111'] = {'mb': ('MSI PRO H610M-E', 8370), 'cpu': ('Intel Core i5 12400F', 14790), 'gpu': ('GIGABYTE NVIDIA GeForce RTX 3060', 36000), 'op': ('Kingston Fury Beast Black DDR5 2x8 5600', 7990)}
budget_pc

In [ ]:
features = 16
# define a simple linear VAE
class LinearVAE(nn.Module):
    def __init__(self):
        super(LinearVAE, self).__init__()

        self.flatten = nn.Flatten()
 
        # encoder
        self.encoder = nn.Sequential(
            nn.Linear(in_features=12288, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=features*2)

        )
 
        # decoder 
        self.decoder = nn.Sequential(
            nn.Linear(in_features=features, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=12288)
        )

    def reparameterize(self, mu, log_var):
        """
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        std = torch.exp(0.5 * log_var) # standard deviation
        eps = torch.randn_like(std) # `randn_like` as we need the same size
        sample = mu + (eps * std) # sampling as if coming from the input space
        return sample
 
    def forward(self, x):
        # encoding
        x = self.flatten(x).float()
        x = self.encoder(x).view(-1, 2, features)
        # get `mu` and `log_var`
        mu = x[:, 0, :] # the first feature values as mean
        log_var = x[:, 1, :] # the other feature values as variance
        # get the latent vector through reparameterization
        z = self.reparameterize(mu, log_var)
 
        # decoding
        x = self.decoder(z)
        reconstruction = torch.sigmoid(x)
        return reconstruction, mu, log_var

    def sample(self, z):
        generated = self.decoder(z)
        generated = torch.sigmoid(generated)
        generated = generated.view(-1, 64, 64, 3)
        return generated

    def get_latent_vector(self, x):
        x = self.flatten(x).float()
        x = self.encoder(x).view(-1, 2, features)
        # get `mu` and `log_var`
        mu = x[:, 0, :] # the first feature values as mean
        log_var = x[:, 1, :] # the other feature values as variance
        # get the latent vector through reparameterization
        z = self.reparameterize(mu, log_var)
        return z